In [5]:
import re

def text_pr(old, new):
    # old, new 둘 다 소문자로 변환
    o = old.lower()
    n = new.lower()

    # 공백과 콤마 제거
    o_clean = re.sub(r"[ ,]", "", o)
    n_clean = re.sub(r"[ ,]", "", n)

    # 공통 prefix 찾기
    i = 0
    while i < len(o_clean) and i < len(n_clean) and o_clean[i] == n_clean[i]:
        i += 1

    # old는 공통 부분까지만, 나머지는 new에서 가져오기
    return new[:i] + new[i:], i

In [8]:
old = "From Now on, we we will try to fix it."
new = "From now on We will try to fix it, okay?"

import time

st = time.time()
text_pr(old, new)
print(time.time() - st)

4.1484832763671875e-05


In [1]:
from vllm import LLM, SamplingParams
model_path = "ByteDance-Seed/Seed-X-PPO-7B"
model = LLM(model=model_path,
            max_num_seqs=512,
            tensor_parallel_size=1,
            enable_prefix_caching=True, 
            gpu_memory_utilization=0.95)

INFO 09-02 10:46:06 [__init__.py:241] Automatically detected platform cuda.
INFO 09-02 10:46:07 [utils.py:326] non-default args: {'model': 'ByteDance-Seed/Seed-X-PPO-7B', 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.95, 'max_num_seqs': 512, 'disable_log_stats': True}
INFO 09-02 10:46:14 [__init__.py:711] Resolved architecture: MistralForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 09-02 10:46:14 [__init__.py:1750] Using max model len 32768
INFO 09-02 10:46:15 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.


ValueError: Kwargs ['max_loras', '_from_auto'] are not supported by `MistralCommonTokenizer.from_pretrained`.

In [ ]:
import time

messages = [
    # without CoT
    "Translate the following English sentence into Korean:\nThe Fed is actually not just the central bank of the United States, but it controls many other banks.<ko>",
    # with CoT
    # "Translate the following English sentence into Korean and explain it in detail:\nMay the force be with you <ko>" 
]

# Beam search (We recommend using beam search decoding)
# decoding_params = BeamSearchParams(beam_width=4,
#                                    max_tokens=512)
# Greedy decoding
decoding_params = SamplingParams(temperature=0.2,
                                 max_tokens=512,
                                 skip_special_tokens=True)

st = time.time()
results = model.generate(messages)
print(time.time() - st)
responses = [res.outputs[0].text.strip() for res in results]
print(responses)